### Donwloading Dataset

In [0]:
!pip install kaggle
!mkdir .kaggle

In [0]:
import json
token = {"username":"ystark","key":"7d9d124dcd4afabb3ef553b00daea011"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
#May require to run twice
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json;
!kaggle config set -n path -v {/content};

In [0]:
!chmod 600 /root/.kaggle/kaggle.json;
!kaggle competitions download -c twitter-sentiment-analysis2;

!unzip /content/{/content}/competitions/twitter-sentiment-analysis2/train.csv.zip;
!unzip /content/{/content}/competitions/twitter-sentiment-analysis2/test.csv.zip;

# Begin Project

file descriptions

train.csv - the training set
test.csv - the test set
Data fields

ItemID - id of twit
Sentiment - sentiment
SentimentText - text of the twit


0 - negative

1 - positive

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import string 
import re
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

In [0]:
train_data=pd.read_csv('/content/train.csv',encoding = "ISO-8859-1")
train_data.head()

### Preprocessing

In [0]:
tweets=pd.DataFrame(train_data.SentimentText)

In [0]:
tt = TweetTokenizer()
tweets['tokenized_text']=tweets.apply(lambda row: tt.tokenize(row['SentimentText']),axis=1)
tweets['joined_prepr_text'] = tweets['tokenized_text'].apply(lambda x: ' '.join(map(str, x)))

In [0]:
tweets.head()

In [0]:
vocab_size=0
for i in range(len(tweets)):
  vocab_size+=len(tweets.tokenized_text[i])

print('Vocab_size: ',vocab_size)

### Create Train-Test Set

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test=train_test_split(tweets['joined_prepr_text'],train_data['Sentiment'],test_size = 0.2,random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [0]:
X_val.shape

In [0]:
print(X_train.shape, X_test.shape)

#### Model Training

In [0]:
def recall(y_true, y_pred):
    
    """
    Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    
    """
    Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    Source
    ------
    https://github.com/fchollet/keras/issues/5400#issuecomment-314747992
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1(y_true, y_pred):
    
    """Calculate the F1 score."""
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r))


def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)), axis=1)

In [0]:
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.layers import Dropout, Dense
from tensorflow.khttps://machinelearningmastery.com/save-load-keras-deep-learning-models/eras.optimizers import Adam
from tensorflow.keras import backend as K

n_classes = 1

model2 = Sequential()
model2.add(Dense(512, input_dim=X_train_cent.shape[1] , activation='relu'))
model2.add(Dropout(0.3))
# model2.add(Dense(1024,  activation='relu'))
# model2.add(Dropout(0.5))
model2.add(Dense(124,  activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(n_classes,  activation='sigmoid'))

print(model2.summary())
model2.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=[precision, recall, f1, accuracy])

checkpoint2 = ModelCheckpoint('keras_fastetxt_centroids_model', monitor='val_f1', verbose=1, save_best_only=True, mode='max')

history2 = model2.fit(X_train_cent, y_train,
              batch_size=32,
              epochs=15,
              verbose = 1,
              callbacks=[checkpoint2],
              validation_data=(X_val_cent, y_val),
              shuffle=True)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt


# summarize history for f1
plt.plot(history2.history['f1'])
plt.plot(history2.history['val_f1'])
plt.title('model f1')
plt.ylabel('f1-score')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper right')
plt.show()

### Results

#### Dummy Classifier


In [0]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

results = {}

base = DummyClassifier(strategy='most_frequent')
base.fit(X_train_tfidf, y_train)

predictions = base.predict(X_train_tfidf)
results['f1-score train'] = f1_score(y_train, predictions)
results['f1-score macro train'] = f1_score(y_train, predictions, average='macro')
results['precision train'] = precision_score(y_train, predictions)
results['recall train'] = recall_score(y_train, predictions)
print("train f1-score: %.2f%%" % (results['f1-score train']*100))
print("train macro f1-score: %.2f%%" % (results['f1-score macro train']*100))
print("train precision: %.2f%%" % (results['precision train']*100))
print("train recall: %.2f%%" % (results['recall train']*100))

predictions_test = base.predict(X_test_tfidf)
results['f1-score test'] = f1_score(y_test, predictions_test)
results['f1-score macro test'] = f1_score(y_test, predictions_test, average='macro')
results['precision test'] = precision_score(y_test, predictions_test)
results['recall test'] = recall_score(y_test, predictions_test)
print("test f1-score: %.2f%%" % (results['f1-score test']*100))
print("test macro f1-score: %.2f%%" % (results['f1-score macro test']*100))
print("test precision: %.2f%%" % (results['precision test']*100))
print("test recall: %.2f%%" % (results['recall test']*100))
print()
print("test data confusion matrix")
y_true = pd.Series(y_test, name='True')
y_pred = pd.Series(predictions_test, name='Predicted')
pd.crosstab(y_true, y_pred)

### MLP

In [0]:
predictions = model2.predict(X_train_cent)
predictions = np.where(predictions>0.5, 1, predictions)
predictions = np.where(predictions<0.5, 0, predictions)
results['f1-score train'] = f1_score(y_train, predictions)
results['f1-score macro train'] = f1_score(y_train, predictions, average='macro')
results['precision train'] = precision_score(y_train, predictions)
results['recall train'] = recall_score(y_train, predictions)
print("train f1-score: %.2f%%" % (results['f1-score train']*100))
print("train macro f1-score: %.2f%%" % (results['f1-score macro train']*100))
print("train precision: %.2f%%" % (results['precision train']*100))
print("train recall: %.2f%%" % (results['recall train']*100))

predictions_test = model2.predict(X_test_cent)
predictions_test = np.where(predictions_test>0.5, 1, predictions_test)
predictions_test = np.where(predictions_test<0.5, 0, predictions_test)
results['f1-score test'] = f1_score(y_test, predictions_test)
results['f1-score macro test'] = f1_score(y_test, predictions_test, average='macro')
results['precision test'] = precision_score(y_test, predictions_test)
results['recall test'] = recall_score(y_test, predictions_test)
print("test f1-score: %.2f%%" % (results['f1-score test']*100))
print("test macro f1-score: %.2f%%" % (results['f1-score macro test']*100))
print("test precision: %.2f%%" % (results['precision test']*100))
print("test recall: %.2f%%" % (results['recall test']*100))
print()
print("test data confusion matrix")
y_true = pd.Series(y_test, name='True')
y_pred = pd.Series(predictions_test.ravel(), name='Predicted')
pd.crosstab(y_true, y_pred)

#### Multinomial NB 

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB(alpha=1.1,fit_prior=True)

#instantiate X_train feature vector according to the best tf-idf parameters fitted for NB
tfidf_NB=TfidfVectorizer(ngram_range=(1,2),max_features = 10000,binary=False,norm='l2',max_df=1.0)
X_train_tfidf_NB=tfidf_NB.fit_transform(X_train)
X_test_tfidf_NB=tfidf_NB.transform(X_test)

NB.fit(X_train_tfidf_NB, y_train)
results = {}

predictions = NB.predict(X_train_tfidf_NB)
results['f1-score train'] = f1_score(y_train, predictions)
results['f1-score macro train'] = f1_score(y_train, predictions, average='macro')
results['precision train'] = precision_score(y_train, predictions)
results['recall train'] = recall_score(y_train, predictions)
print("train f1-score: %.2f%%" % (results['f1-score train']*100))
print("train macro f1-score: %.2f%%" % (results['f1-score macro train']*100))
print("train precision: %.2f%%" % (results['precision train']*100))
print("train recall: %.2f%%" % (results['recall train']*100))

predictions_test = NB.predict(X_test_tfidf_NB)
results['f1-score test'] = f1_score(y_test, predictions_test)
results['f1-score macro test'] = f1_score(y_test, predictions_test, average='macro')
results['precision test'] = precision_score(y_test, predictions_test)
results['recall test'] = recall_score(y_test, predictions_test)
print("test f1-score: %.2f%%" % (results['f1-score test']*100))
print("test macro f1-score: %.2f%%" % (results['f1-score macro test']*100))
print("test precision: %.2f%%" % (results['precision test']*100))
print("test recall: %.2f%%" % (results['recall test']*100))
print()
print("test data confusion matrix")
y_true = pd.Series(y_test, name='True')
y_pred = pd.Series(predictions_test, name='Predicted')
pd.crosstab(y_true, y_pred)

### Learning Curves 

In [0]:
#Learning curves with cross-validtion
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
   
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("F1-score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs,scoring='f1', train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="b")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="orange")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="b",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="orange",
             label="Cross-validation score")

    plt.legend(loc="lower right")
    return plt


X, y = X_train_tfidf_LR,y_train


title = "Learning Curves (Logistic Regression)"
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
plot_learning_curve(lr, title, X_train_tfidf_LR, y, (0.7, 1.01), cv=cv, n_jobs=-1)
plt.show()


title = "Learning Curves (Naive Bayes)"
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
# estimator = MultinomialNB()
plot_learning_curve(NB, title, X_train_tfidf_NB, y, ylim=(0.7, 1.01), cv=cv, n_jobs=2)
plt.show()

In [0]:
#Precision-Recall curves
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc


#disable warnings for large float numbers of estimated probabilities
np.seterr(all='ignore')
estimators = {'Logistic Regression':lr,
              'Naive Bayes':NB}

for (name,estimator) in estimators.items():
    
    model =estimator
    if(name=='Logistic Regression'):
      pred = model.predict_proba(X_test_tfidf_LR)
    elif(name=='Naive Bayes'):
      pred = model.predict_proba(X_test_tfidf_NB)
      
    precision, recall, thresholds = precision_recall_curve(y_test, pred[:,1])
    area = auc(recall, precision)

    plt.plot(recall, precision, label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall %s: AUC=%0.2f' % (name,area))
    plt.legend(loc="lower left")
    plt.show()

### Bootstrap

In [0]:
from tqdm import tqdm
from sklearn.utils import resample

def bootstrap_test(n_iterations, n_size, X, y, model1, model2):
  #this function was based on slide 31
  #get predictions on the whole test set for the two models
  initial_predictions_model1 = model1.predict(X)
  initial_predictions_model2 = model2.predict(X)
  initial_model1_f1score = f1_score(y, initial_predictions_model1, average='macro')
  initial_model2_f1score = f1_score(y, initial_predictions_model2, average='macro')
  #estimate δ(x) for whole test set
  initial_dif = initial_model1_f1score - initial_model2_f1score
  #setup b
  bsets = 0
  my_list=list(range(n_iterations))
  for i in tqdm(my_list):
    #sample with replacement the test set
    X_sub, y_sub = resample(X, y, n_samples=n_size)
    #get predictions on the new sub-set for the two models
    subsample_predictions_model1 = model1.predict(X_sub)
    subsample_predictions_model2 = model2.predict(X_sub)
    subsample_model1_f1score = f1_score(y_sub, subsample_predictions_model1, average='macro')
    subsample_model2_f1score = f1_score(y_sub, subsample_predictions_model2, average='macro')
    #estimate δ(x*(i)) for new sub-set test set
    subsample_dif = subsample_model1_f1score - subsample_model2_f1score
    if subsample_dif > 2 * initial_dif:
      bsets +=1
    p_value = bsets / n_iterations
  return p_value

In [0]:
# configure bootstrap
n_iterations = 10000
n_size = len(y_test)

In [0]:
# run bootstrap between LR and dummy classifier
bootstrap_test(n_iterations, n_size, X_test_tfidf_LR, y_test, lr, base)

In [0]:
# run bootstrap between LR and Naive Bayes
bootstrap_test(n_iterations, n_size, X_test_tfidf_LR, y_test, lr, NB)